<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/gan/t_dcgan_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Generative Adversarial Network



## What are GANs?
[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661) (GANs) are one of the most interesting ideas in computer science today. Two models are trained simultaneously by an adversarial process. A *generator* ("the artist") learns to create images that look real, while a *discriminator* ("the art critic") learns to tell real images apart from fakes.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/gan1.png" alt="A diagram of a generator and discriminator" width="450" height="350">

During training, the *generator* progressively becomes better at creating images that look real, while the *discriminator* becomes better at telling them apart. The process reaches equilibrium when the *discriminator* can no longer distinguish real images from fakes.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/gan2.png" alt="A second diagram of a generator and discriminator" width="450" height="350">

This notebook demonstrates this process on the MNIST dataset. The following animation shows a series of images produced by the *generator* as it was trained for 50 epochs. The images begin as random noise, and increasingly resemble hand written digits over time.

![sample output](https://tensorflow.org/images/gan/dcgan.gif)

## Setup

In [2]:
!pip install imageio -q
!pip install git+https://github.com/tensorflow/docs -q

In [3]:
import glob
import imageio
import os
import PIL
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from IPython import display

tf.__version__, imageio.__version__

('2.3.0', '2.4.1')